# Run the following cell without changing anything

In [ ]:
# example of using saved cycleGAN models for image translation
#based on https://machinelearningmastery.com/cyclegan-tutorial-with-keras/
from keras.models import load_model
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
import os
import numpy as np
import tifffile
from scipy.ndimage import zoom
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from skimage.measure import label, regionprops

def auximread(filepath):
    image = tifffile.imread(filepath)
    #the output image should be (X,Y,Z)
    original_0 = np.shape(image)[0]
    original_1 = np.shape(image)[1]
    original_2 = np.shape(image)[2]
    index_min = np.argmin([original_0, original_1, original_2])
    if index_min == 0:
        image = image.transpose(1,2,0)
    elif index_min == 1:
        image = image.transpose(0,2,1)
    return image

def predict(model_dir, img_dir, save_dir_2dmasks, save_dir_3dmasks, path_res, _resize, norm_, mode_, patch_size, _step, _step_z=32, _patch_size_z=64):

    #load the file with resolution information and extract features
    resolution = pd.read_excel(path_res)

    cust = {'InstanceNormalization': InstanceNormalization}
    #load the model
    model_BtoA = load_model(model_dir, cust)

    print('Mode: {}'.format(mode_))

    _patch_size = patch_size[1]
    _nbslices = patch_size[0]

    for img_name in os.listdir(img_dir):

        aux_res = resolution[resolution['Image'] == img_name]
        dim_x = aux_res['resx'].values[0]
        dim_y = aux_res['resy'].values[0]
        dim_z = aux_res['resz'].values[0]
        perceqmin = aux_res['perceqmin'].values[0]
        perceqmax = aux_res['perceqmax'].values[0]


        print('Image Name: {}'.format(img_name))
        image = auximread(os.path.join(img_dir, img_name))

        image = ((image/(np.max(image)))*255).astype('uint8')

        print('Image Shape: {}'.format(image.shape))
        print('----------------------------------------')

        initial_image_x = np.shape(image)[0]
        initial_image_y = np.shape(image)[1]
        initial_image_z = np.shape(image)[2]

        #percentile equalization
        if norm_:
            minval = np.percentile(image, perceqmin) 
            maxval = np.percentile(image, perceqmax)
            image = np.clip(image, minval, maxval)
            image = (((image - minval) / (maxval - minval)) * 255).astype('uint8')

        if _resize:
            image = zoom(image, (dim_x/0.333, dim_y/0.333, dim_z/0.5), order=3, mode='nearest')
            image = ((image/np.max(image))*255.0).astype('uint8')


        #image size
        size_y = np.shape(image)[0]
        size_x = np.shape(image)[1]
        size_depth = np.shape(image)[2]
        aux_sizes_or = [size_y, size_x, size_depth]
        

        #patch size
        new_size_y = int((size_y/_patch_size) + 1) * _patch_size
        new_size_x = int((size_x/_patch_size) + 1) * _patch_size
        new_size_z = int((size_depth/_patch_size_z) + 1) * _patch_size_z
        aux_sizes = [new_size_y, new_size_x, new_size_z]
        
        ## zero padding
        aux_img = np.random.randint(1,50,(aux_sizes[0], aux_sizes[1], aux_sizes[2]))
        aux_img[0:aux_sizes_or[0], 0:aux_sizes_or[1],0:aux_sizes_or[2]] = image
        image = aux_img
        del aux_img
            
        final_mask_foreground = np.zeros((np.shape(image)[0], np.shape(image)[1], np.shape(image)[2]))
        final_mask_background = np.zeros((np.shape(image)[0], np.shape(image)[1], np.shape(image)[2]))
        final_mask_background = final_mask_background.astype('uint8')
        final_mask_foreground = final_mask_foreground.astype('uint8')
        

        total_iterations = int(image.shape[0]/_patch_size)

        with tqdm(total=total_iterations) as pbar:
            i=0
            while i+_patch_size<=image.shape[0]:
                j=0
                while j+_patch_size<=image.shape[1]:
                    k=0
                    while k+_patch_size_z<=image.shape[2]:
                    
                        B_real = np.zeros((1,_nbslices,_patch_size,_patch_size,1),dtype='float32')
                        _slice = image[i:i+_patch_size, j:j+_patch_size, k:k+_patch_size_z]
                        
                        _slice = _slice.transpose(2,0,1)
                        _slice = np.expand_dims(_slice, axis=-1)
            
                        B_real[0,:]=(_slice-127.5) /127.5   
            
                        A_generated  = model_BtoA.predict(B_real)
            
                        A_generated = (A_generated + 1)/2 #from [-1,1] to [0,1]
            
                        A_generated = A_generated[0,:,:,:,0]
                        A_generated = A_generated.transpose(1,2,0)
            
                        #print(np.unique(A_generated))
                        A_generated = (A_generated>0.5)*1
            
                        A_generated = A_generated.astype('uint8')
            
                        final_mask_foreground[i:i+_patch_size, j:j+_patch_size, k:k+_patch_size_z] = final_mask_foreground[i:i+_patch_size, j:j+_patch_size, k:k+_patch_size_z] + A_generated
                        final_mask_background[i:i+_patch_size, j:j+_patch_size, k:k+_patch_size_z] = final_mask_background[i:i+_patch_size, j:j+_patch_size, k:k+_patch_size_z] + (1-A_generated)
                        
                        k=k+_step_z
                    j=j+_step
                i=i+_step
                pbar.update(1)


        del _slice
        del A_generated
        del B_real

        final_mask = (final_mask_foreground>=final_mask_background)*1

        image = image[0:aux_sizes_or[0], 0:aux_sizes_or[1],0:size_depth]
        print('Image Shape: {}'.format(image.shape))
        print('----------------------------------------')

        final_mask = final_mask[0:aux_sizes_or[0], 0:aux_sizes_or[1],0:aux_sizes_or[2]]


        if _resize:
            final_mask = zoom(final_mask, (0.333/dim_x, 0.333/dim_y, 0.5/dim_z), order=3, mode='nearest')
            final_mask = (final_mask*255.0).astype('uint8')

            final_size_x = np.shape(final_mask)[0]
            final_size_y = np.shape(final_mask)[1]
            final_size_z = np.shape(final_mask)[2]

            aux_mask = np.zeros((initial_image_x, initial_image_y, initial_image_z)).astype('uint8')
            aux_mask[0:min(initial_image_x, final_size_x),0:min(initial_image_y, final_size_y),0:min(initial_image_z, final_size_z)] = final_mask[0:min(initial_image_x, final_size_x),0:min(initial_image_y, final_size_y),0:min(initial_image_z, final_size_z)]

            final_mask = aux_mask.copy()


        print('Mask Shape: {}'.format(final_mask.shape))
        print('----------------------------------------')
        final_mask = final_mask/np.max(final_mask)
        final_mask = final_mask*255.0
        final_mask = final_mask.astype('uint8')
        tifffile.imwrite(os.path.join(save_dir_3dmasks, img_name), final_mask)
        seg = np.max(final_mask, axis=2) 
        tifffile.imwrite(os.path.join(save_dir_2dmasks, img_name), seg)

        del seg
        del final_mask
        del image
        

def post_processing(msk_dir, save_dir_masks3d, save_dir_masks2d, resolution_file):
    resolution = pd.read_excel(resolution_file)
    for msk_name in os.listdir(msk_dir):
        if '.tif' in msk_name:
            #print('here: {}'.format(msk_name))
            final_mask = tifffile.imread(os.path.join(msk_dir, msk_name))
            final_mask = final_mask/255.0
            final_mask = final_mask.astype('uint8')

            aux_res = resolution[resolution['Image'] == msk_name]
            dim_x = aux_res['resx'].values[0]
            dim_y = aux_res['resy'].values[0]
            dim_z = aux_res['resz'].values[0]

            ##post-processing
            #remove outliers
            print('Remove the Outliers')
            labeled_image = label(final_mask)
            properties = regionprops(labeled_image)

            mask = np.zeros(np.shape(final_mask)).astype('uint8')

            for prop in properties:
                area_ = prop.area*(dim_x*dim_y*dim_z)
                #print(area_)
                if area_ < 50000 and area_ >= 1000:
                    conv = prop.area/prop.area_convex
                    if conv<=0.2:
                        mask[labeled_image==prop.label] = 255 #region to include
                elif area_ > 50000:
                    mask[labeled_image==prop.label] = 255 #region to include

            final_mask = mask.copy()
            final_mask = final_mask.astype('uint8')

            del mask
            del properties
            print('Objects Removed')


            print('Mask Shape: {}'.format(final_mask.shape))
            print('----------------------------------------')
            tifffile.imwrite(os.path.join(save_dir_masks3d, msk_name), final_mask)

            seg = np.max(final_mask, axis=2)  
            tifffile.imwrite(os.path.join(save_dir_masks2d, msk_name), seg)
            
            
#!/bin/env python

##
## calculate the concave hull of a set of points
## see: CONCAVE HULL: A K-NEAREST NEIGHBOURS APPROACH
##      FOR THE COMPUTATION OF THE REGION OCCUPIED BY A
##      SET OF POINTS
##      Adriano Moreira and Maribel Yasmina Santos 2007
##

import numpy as np
import scipy.spatial as spt
import matplotlib

def doBoundingBoxesIntersect(a, b, c, d):
    '''
    Check if bounding boxes do intersect. If one bounding box touches
    the other, they do intersect.
    First segment is of points a and b, second of c and d.
    '''
    ll1_x = min(a[0],b[0]); ll2_x = min(c[0],d[0])
    ll1_y = min(a[1],b[1]); ll2_y = min(c[1],d[1])
    ur1_x = max(a[0],b[0]); ur2_x = max(c[0],d[0])
    ur1_y = max(a[1],b[1]); ur2_y = max(c[1],d[1])

    return ll1_x <= ur2_x and \
           ur1_x >= ll2_x and \
           ll1_y <= ur2_y and \
           ur1_y >= ll2_y

def isPointOnLine(a,b,c):
    '''
    Check if a point is on a line.
    '''
    # move to origin
    aTmp = (0,0)
    bTmp = (b[0] - a[0], b[1] - a[1])
    cTmp = (c[0] - a[0], c[1] - a[1])
    r = np.cross(bTmp, cTmp)
    return np.abs(r) < 0.0000000001

def isPointRightOfLine(a,b,c):
    '''
    Check if a point (c) is right of a line (a-b).
    If (c) is on the line, it is not right it.
    '''
    # move to origin
    aTmp = (0,0)
    bTmp = (b[0] - a[0], b[1] - a[1])
    cTmp = (c[0] - a[0], c[1] - a[1])
    return np.cross(bTmp, cTmp) < 0

def lineSegmentTouchesOrCrossesLine(a,b,c,d):
    '''
    Check if line segment (a-b) touches or crosses
    line segment (c-d).
    '''
    return isPointOnLine(a,b,c) or \
           isPointOnLine(a,b,d) or \
          (isPointRightOfLine(a,b,c) ^
           isPointRightOfLine(a,b,d))

def doLinesIntersect(a,b,c,d):
    '''
    Check if line segments (a-b) and (c-d) intersect.
    '''
    return doBoundingBoxesIntersect(a,b,c,d) and \
           lineSegmentTouchesOrCrossesLine(a,b,c,d) and \
           lineSegmentTouchesOrCrossesLine(c,d,a,b)


##############################
## Tests
##############################

def test_doBoundingBoxesIntersect():
    A=(1,1); B=(2,2); C=(3,1); D=(4,2)
    assert doBoundingBoxesIntersect(A,B,C,D) == False
    A=(1,2); B=(3,4); C=(2,1); D=(4,3)
    assert doBoundingBoxesIntersect(A,B,C,D) == True

def test_isPointOnLine():
    A=(1,1); B=(3,3); C=(2,2)
    assert isPointOnLine(A,B,C) == True
    A=(1,1); B=(3,3); C=(3,2)
    assert isPointOnLine(A,B,C) == False


def test_isPointRightOfLine():
    A=(1,1); B=(3,3); C=(2,2)
    assert isPointRightOfLine(A,B,C) == False
    A=(1,1); B=(3,3); C=(3,2)
    assert isPointRightOfLine(A,B,C) == True
    A=(1,1); B=(3,3); C=(1,2)
    assert isPointRightOfLine(A,B,C) == False

# a lot of testcases to be tested with the final function

def tcase(name):
    if name == 'F1':
        return (0,0), (7,7), (3,4), (4,5), False
    elif name == 'F2':
        return (-4,4), (-2,1), (-2,3), (0,0), False
    elif name == 'F3':
        return (0,0), (0,1), (2,2), (2,3), False
    elif name == 'F4':
        return (0,0), (0,1), (2,2), (3,2), False
    elif name == 'F5':
        return (-1,-1), (2,2), (3,3), (5,5), False
    elif name == 'F6':
        return (0,0), (1,1), (2,0), (0.5,2), False
    elif name == 'F7':
        return (1,1), (4,1), (2,2), (3,2), False
    elif name == 'F8':
        return (0,5), (6,0), (2,1), (2,2), False
    elif name == 'T1':
        return (0,-2), (0,2), (-2,0), (2,0), True
    elif name == 'T2':
        return (5,5), (0,0), (1,1), (8,2), True
    elif name == 'T3':
        return (-1,0), (0,0), (-1,-1), (-1,1), True
    elif name == 'T4':
        return (0,2), (2,2), (2,0), (2,4), True
    elif name == 'T5':
        return (0,0), (5,5), (1,1), (3,3), True
    elif name == 'T6':
        return (0,0), (3,3), (0,0), (3,3), True

cases = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8',
         'T1', 'T2', 'T3', 'T4', 'T5', 'T6']

def check_intersection(name):
    A,B,C,D, result = tcase(name)
    assert doLinesIntersect(A,B,C,D) == result

def test_doLinesIntersect():
    for case in cases:
        yield check_intersection, case


def GetFirstPoint(dataset):
    ''' Returns index of first point, which has the lowest y value '''
    # todo: what if there is more than one point with lowest y?
    imin = np.argmin(dataset[:,1])
    return dataset[imin]

def GetNearestNeighbors(dataset, point, k):
    ''' Returns indices of k nearest neighbors of point in dataset'''
    # todo: experiment with leafsize for performance
    mytree = spt.cKDTree(dataset,leafsize=10)
    distances, indices = mytree.query(point,k)
    # todo: something strange here, we get more indices than points in dataset
    #       so have to do this
    return dataset[indices[:dataset.shape[0]]]

def SortByAngle(kNearestPoints, currentPoint, prevPoint):
    ''' Sorts the k nearest points given by angle '''
    angles = np.zeros(kNearestPoints.shape[0])
    i = 0
    for NearestPoint in kNearestPoints:
        # calculate the angle
        angle = np.arctan2(NearestPoint[1]-currentPoint[1],
                NearestPoint[0]-currentPoint[0]) - \
                np.arctan2(prevPoint[1]-currentPoint[1],
                prevPoint[0]-currentPoint[0])
        angle = np.rad2deg(angle)
        # only positive angles
        angle = np.mod(angle+360,360)
        #print NearestPoint[0], NearestPoint[1], angle
        angles[i] = angle
        i=i+1
    return kNearestPoints[np.argsort(angles)]

def plotPoints(dataset):
    matplotlib.pyplot.plot(dataset[:,0],dataset[:,1],'o',markersize=10,markerfacecolor='0.75',
            markeredgewidth=1)
    matplotlib.pyplot.axis('equal')
    matplotlib.pyplot.axis([min(dataset[:,0])-0.5,max(dataset[:,0])+0.5,min(dataset[:,1])-0.5,
        max(dataset[:,1])+0.5])
    matplotlib.pyplot.show()

def plotPath(dataset, path):
    matplotlib.pyplot.plot(dataset[:,0],dataset[:,1],'o',markersize=10,markerfacecolor='0.65',
            markeredgewidth=0)
    path = np.asarray(path)
    matplotlib.pyplot.plot(path[:,0],path[:,1],'o',markersize=10,markerfacecolor='0.55',
            markeredgewidth=0)
    matplotlib.pyplot.plot(path[:,0],path[:,1],'-',lw=1.4,color='k')
    matplotlib.pyplot.axis('equal')
    matplotlib.pyplot.axis([min(dataset[:,0])-0.5,max(dataset[:,0])+0.5,min(dataset[:,1])-0.5,
        max(dataset[:,1])+0.5])
    matplotlib.pyplot.axis('off')
    matplotlib.pyplot.savefig('./doc/figure_1.png', bbox_inches='tight')
    matplotlib.pyplot.show()

def removePoint(dataset, point):
    delmask = [np.logical_or(dataset[:,0]!=point[0],dataset[:,1]!=point[1])]
    newdata = dataset[tuple(delmask)]
    return newdata


def concaveHull(dataset, k):
    assert k >= 3, 'k has to be greater or equal to 3.'
    points = dataset
    # todo: remove duplicate points from dataset
    # todo: check if dataset consists of only 3 or less points
    # todo: make sure that enough points for a given k can be found

    firstpoint = GetFirstPoint(points)
    # init hull as list to easily append stuff
    hull = []
    # add first point to hull
    hull.append(firstpoint)
    # and remove it from dataset
    points = removePoint(points,firstpoint)
    currentPoint = firstpoint
    # set prevPoint to a Point righ of currentpoint (angle=0)
    prevPoint = (currentPoint[0]+10, currentPoint[1])
    step = 2

    while ( (not np.array_equal(firstpoint, currentPoint) or (step==2)) and points.size > 0 ):
        if ( step == 5 ): # we're far enough to close too early
            points = np.append(points, [firstpoint], axis=0)
        kNearestPoints = GetNearestNeighbors(points, currentPoint, k)
        cPoints = SortByAngle(kNearestPoints, currentPoint, prevPoint)
        # avoid intersections: select first candidate that does not intersect any
        # polygon edge
        its = True
        i = 0
        while ( (its==True) and (i<cPoints.shape[0]) ):
                i=i+1
                if ( np.array_equal(cPoints[i-1], firstpoint) ):
                    lastPoint = 1
                else:
                    lastPoint = 0
                j = 2
                its = False
                while ( (its==False) and (j<np.shape(hull)[0]-lastPoint) ):
                    its = doLinesIntersect(hull[step-1-1], cPoints[i-1],
                            hull[step-1-j-1],hull[step-j-1])
                    j=j+1
        if k>50:
            return [[0,0],[0,0]]
        if ( its==True ):
            print("all candidates intersect -- restarting with k = ",k+1)
            return concaveHull(dataset,k+1)
        prevPoint = currentPoint
        currentPoint = cPoints[i-1]
        # add current point to hull
        hull.append(currentPoint)
        points = removePoint(points,currentPoint)
        step = step+1
    # check if all points are inside the hull
    p = matplotlib.path.Path(hull)
    pContained = p.contains_points(dataset, radius=0.0000000001)
    if (not pContained.all()):
        print ("not all points of dataset contained in hull -- restarting with k = ",k+1)
        return concaveHull(dataset, k+1)

    print ("finished with k = ",k)
    return hull


In [ ]:
pip show tensorflow keras


# Perform Image Segmentation

#### Change the directories and run the following cell, it will save 2D projections of the output segmentation masks in the folder "save_dir_2dmasks" and the output 3D masks in the folder "save_dir_3dmasks"

In [ ]:
model_dir = '/home/jovyan/3dvascnet/CycleGANVesselSegmentation.h5' #path to the model .h5 file (download at https://huggingface.co/Hemaxi/3DCycleGAN/tree/main)
img_dir = r'/home/jovyan/3dvascnet/images' #directory containing the .tif files
save_dir_2dmasks = r'/home/jovyan/3dvascnet/results2d' #output 2d projections of the 3d masks
save_dir_3dmasks = r'/home/jovyan/3dvascnet/results3d' #output 3d masks
path_res = r'/home/jovyan/3dvascnet/resolution.xlsx' #path to the resolution file
mode_ = 'test'
_step = 64
norm_ = True #percentile based normalization
_resize = True #image resizing to match the resolution of the training images
patch_size = (64,128,128,1)

predict(model_dir, img_dir, save_dir_2dmasks, save_dir_3dmasks, path_res, _resize, norm_, mode_,  patch_size, _step)

#### Change the directories and run the following cell, it will post-process the masks

In [ ]:
save_dir_masks2d_postproc = r'/home/jovyan/3dvascnet/results2d_postproc' #post-processed masks in 2D
save_dir_masks3d_postproc = r'/home/jovyan/3dvascnet/results3d_postproc' #post-processed masks in 3D

post_processing(save_dir_3dmasks, save_dir_masks3d_postproc, save_dir_masks2d_postproc, path_res)

# Perform Quantification

#### Run the code below, it will quantify the vasculature features for the 3D output masks, a features3d.csv file will be generated containing the computed 3D features for each mask, it will have the following structure:

```
| Image        | Group | Branching Points Density | Vessel Density % | Avascular Area % | Mean Branch Length | Mean Vessel Radius |
|--------------|-------|--------------------------|------------------|------------------|--------------------|--------------------|
| Image_Name_1 |       |                          |                  |                  |                    |                    |  
| Image_Name_2 |       |                          |                  |                  |                    |                    |    
| Image_Name_3 |       |                          |                  |                  |                    |                    |    
```

In [ ]:
#3D Retinal Vasculature Quantification
import pandas as pd
from skimage.morphology import binary_erosion, binary_dilation, convex_hull_image, skeletonize_3d, label
from skimage import draw
from skimage.measure import regionprops
import cv2
from skan import Skeleton, summarize
import numpy as np
import os
from scipy import signal
import tifffile
import scipy.ndimage as ndi
from PIL import Image, ImageDraw
from scipy.sparse.csgraph import shortest_path
from skan.csr import skeleton_to_csgraph
from scipy import ndimage

masks_dir = save_dir_masks3d_postproc
resolution_file = path_res

resolution = pd.read_excel(resolution_file)

#Function for the computation of the number of bifurcations and end points
def branching_points(branch_data):
    branch_points=0
    end_points=0
    node_origin = branch_data['node-id-src'].values.tolist()
    node_destination = branch_data['node-id-dst'].values.tolist()
    all_nodes = node_origin + node_destination
    aux_list = np.array(all_nodes)
    bpoints_id = []
    epoints_id = []
    for element in np.unique(aux_list):
        occur = all_nodes.count(element)
        if occur>=3:
            branch_points=branch_points+1
            bpoints_id.append(element)
        elif occur==1:
            end_points=end_points+1
            epoints_id.append(element)
    return branch_points, bpoints_id, end_points, epoints_id


def radius_3d_main(skeleton, branch_data, mask, boundary):
    pixel_graph, coordinates_ = skeleton_to_csgraph(skeleton, spacing=[1,1,1])
    dist_matrix, predecessors = shortest_path(pixel_graph, directed=True, indices=branch_data['node-id-src'].to_numpy(), return_predecessors=True)
    #dist_matrix has size (#sourceids as in len(branch_data['node-id-src']), #all nodes in the skeleton)
    
    ## iterate through each branch and check the direction
    all_major = []
    all_minor = []
    all_radii = []

    for i in range(len(branch_data)):
        
        #node indices (i is the node-id-src, because it was used before to compute dist_matrix and predecessors)
        b =  int(branch_data.iloc[i]['node-id-dst']) 

        # Check if there is a path between the two nodes (a and b)
        if np.isinf(dist_matrix[i, b]):
            print("No path exists between node a and node b.")
            continue
        else:
            # Reconstruct the path from a to b
            path = [(coordinates_[0][b], coordinates_[1][b], coordinates_[2][b])]
            b = predecessors[i, b]
            while b >= 0:
                path.insert(0, (coordinates_[0][b], coordinates_[1][b], coordinates_[2][b]))
                b = predecessors[i, b]

            path = np.asarray(path)
            #print("Shortest path:", path)        

            #compute the direction of the branch
            delta_x = (branch_data.iloc[i]['image-coord-src-0'])-(branch_data.iloc[i]['image-coord-dst-0'])
            delta_y = (branch_data.iloc[i]['image-coord-src-1'])-(branch_data.iloc[i]['image-coord-dst-1'])
            delta_z = (branch_data.iloc[i]['image-coord-src-2'])-(branch_data.iloc[i]['image-coord-dst-2'])

            direction_unit = np.asarray([delta_x, delta_y, delta_z])
            direction_unit = direction_unit / np.linalg.norm(direction_unit)

            major_axes, minor_axes, radii = compute_radii_aux(path, mask, boundary, direction_unit)

            all_major = all_major + major_axes
            all_minor = all_minor + minor_axes
            all_radii = all_radii + radii
                
    return np.asarray(all_major), np.asarray(all_minor), np.asarray(all_radii)

def extract_2d_slice(segmentation_mask, boundary, point, direction_unit, radius=20):
    D = np.dot(direction_unit, point)
    min_point = np.maximum(point - radius, [0, 0, 0])
    max_point = np.minimum(point + radius, segmentation_mask.shape)
    
    grid_x, grid_y, grid_z = np.meshgrid(
        np.arange(min_point[0], max_point[0]),
        np.arange(min_point[1], max_point[1]),
        np.arange(min_point[2], max_point[2]),
        indexing='ij')
    
    voxel_centers = np.column_stack((grid_x.ravel(), grid_y.ravel(), grid_z.ravel()))
    distances = np.abs(np.dot(voxel_centers, direction_unit) - D)
    plane = (distances < 0.5).reshape(grid_x.shape)
    
    out_mask = np.zeros(plane.shape)
    out_mask = np.logical_and(plane, segmentation_mask[min_point[0]:max_point[0], min_point[1]:max_point[1], min_point[2]:max_point[2]])
    
    out_boundary = np.zeros(boundary.shape)
    out_boundary = np.logical_and(plane, boundary[min_point[0]:max_point[0], min_point[1]:max_point[1], min_point[2]:max_point[2]])
    
    new_point = point * (min_point == 0) + radius * (min_point != 0)

    return out_boundary, out_mask, new_point

def compute_radii_aux(path, mask, boundary, direction_unit):
    major_axes = []
    minor_axes = []
    radii = []
    tam_ = np.shape(path)[0]
    first_point = True
    for p in range(int(tam_/2), tam_):

        point = np.asarray(path[p])

        aux_boundary, aux_mask, point = extract_2d_slice(mask, boundary, point, direction_unit)

        aux_mask = label(aux_mask)
        
        l = aux_mask[point[0], point[1], point[2]]
        
        aux_boundary[aux_mask!=l] = 0
        
        indices_ = np.argwhere(aux_boundary) # get indices of the contour

        if np.shape(indices_)[0]>0:

            all_distances = np.sqrt(np.sum((indices_ - point)**2, axis=-1)) #Euclidean distance
            #from the point to each point in the boundary

            major_curr = np.max(all_distances)
            minor_curr = np.min(all_distances)
            
            if first_point:
                major_axes.append(major_curr)
                minor_axes.append(minor_curr)
                radii.append(np.mean(all_distances))
                first_point = False
            else:
                delta_radius_major = np.abs(major_curr-major_axes[-1])
                delta_radius_minor = np.abs(minor_curr-minor_axes[-1])
                if delta_radius_major<4:
                    major_axes.append(major_curr)
                    minor_axes.append(minor_curr)
                    radii.append(np.mean(all_distances))
                else:
                    break
            
    path = np.flip(path,0)
    for p in range(int(tam_/2), tam_):

        point = np.asarray(path[p])

        aux_boundary, aux_mask, point = extract_2d_slice(mask, boundary, point, direction_unit)

        aux_mask = label(aux_mask)
        
        l = aux_mask[point[0], point[1], point[2]]
        
        aux_boundary[aux_mask!=l] = 0
        
        indices_ = np.argwhere(aux_boundary) # get indices of the contour

        if np.shape(indices_)[0]>0:

            all_distances = np.sqrt(np.sum((indices_ - point)**2, axis=-1)) #Euclidean distance
            #from the point to each point in the boundary
            
            major_curr = np.max(all_distances)
            minor_curr = np.min(all_distances)
            
            if first_point:
                major_axes.append(major_curr)
                minor_axes.append(minor_curr)
                radii.append(np.mean(all_distances))
                first_point = False
            else:
                delta_radius_major = np.abs(major_curr-major_axes[-1])
                delta_radius_minor = np.abs(minor_curr-minor_axes[-1])
                if delta_radius_major<4:
                    major_axes.append(major_curr)
                    minor_axes.append(minor_curr)
                    radii.append(np.mean(all_distances))
                else:
                    break

    return major_axes, minor_axes, radii

def compute_chull(mask):
    ry = 100
    rx = 100
    #mask = tifffile.imread(msk)
    mask = np.max(mask, axis=-1)
    print('mask shape: {}'.format(np.shape(mask)))
    x,y = np.shape(mask)
    mask = cv2.resize(mask, (ry,rx)) #resize the mask to decrease computational cost
    mask[mask!=0] = 255.0
    mask_edges = mask - (ndimage.morphology.binary_erosion(mask)*255.0) #obtain the edges to compute concave hull

    #get the coordinates (x,y) of the points belonging to the edges of the mask
    rows, cols = np.where(mask_edges == 255)
    cols = np.expand_dims(cols, axis=-1)
    rows = np.expand_dims(rows, axis=-1)
    points_2d = np.concatenate((cols, rows), axis=-1)

    #compute the concaveHull
    hull = concaveHull(points_2d,5)   #https://github.com/sebastianbeyer/concavehull

    #convert the points into a binary mask (chull)
    polygon = []
    for i in range(np.shape(hull)[0]):
        polygon.append(hull[i][0])
        polygon.append(hull[i][1])

    if sum(polygon)==0:
        chull = convex_hull_image(mask)
        final_chull = ((chull/np.max(chull))*1).astype('uint8')
    else:
        img = Image.new('L', (ry,rx), 0)
        ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
        chull = np.array(img)

        #resize to the original size
        chull = cv2.resize(chull,(y,x))

        #save the concave hull
        chull = (chull*255.0).astype('uint8')

        #invert the convex hull for post-processing
        chull = 255.0 - chull

        properties = regionprops(label(chull))

        a=[]
        for r in properties:
            a.append(r.area)

        max_ = max(a)

        properties = [a for a in properties if a.area >= 0.05*max_]
        ## remove small regions from the mask as well
        chull_proc = np.zeros(np.shape(chull))
        for r in properties:
            chull_proc[r._label_image==r.label]=1

        #erode the processed chull
        chull_proc = chull_proc.astype('uint8')
        kernel = np.ones((55,55),np.uint8)
        chull_proc = cv2.erode(chull_proc,kernel,iterations = 1)

        #invert again
        chull_proc = 1-chull_proc
        chull_proc = (chull_proc*255.0).astype('uint8')

        #median filter to reduce staircase like borders
        final_chull = signal.medfilt2d(chull_proc, 55)
    
        final_chull = final_chull/np.max(final_chull)
        final_chull = (final_chull*1).astype('uint8')

    return final_chull

ellipsoid = draw.ellipsoid(9,9,3, spacing=(1,1,1), levelset=False)
ellipsoid = ellipsoid.astype('uint8')
ellipsoid = ellipsoid[1:-1,1:-1,1:-1]

vessel_features = pd.DataFrame(columns=('Image', 'Group', 'Branching Points Density',
                                  'Vessel Density %', 'Avascular Area %', 
                                  'Mean Branch Length', 'Mean Vessel Radius'))

i=0

for msk_name in os.listdir(masks_dir):
    print('Mask Name: {}'.format(msk_name))
    
    #get the resolution information
    aux_res = resolution[resolution['Image'] == msk_name]
    dimx = aux_res['resx'].values[0]
    dimy = aux_res['resy'].values[0]
    dimz = aux_res['resz'].values[0]
    print('dimensions: {} {} {}'.format(dimx, dimy, dimz))

    #mask and region of interest (ROI)
    mask = tifffile.imread(os.path.join(masks_dir, msk_name))
    mask[mask!=0] = 1
    mask = mask.astype('uint8')

    #perform 3D morphological closing operation on the mask
    mask = binary_dilation(mask, ellipsoid)
    mask = binary_erosion(mask, ellipsoid)
    mask = mask*1
    mask = mask.astype('uint8')
    
    print('Mask Shape: {}'.format(np.shape(mask)))

    chull = compute_chull(mask)

    # Perform resampling using cubic interpolation
    mask = ndi.zoom(mask, (dimx, dimy, dimz), order=3, mode='nearest')

    chull = cv2.resize(chull, (np.shape(mask)[1], np.shape(mask)[0]))

    ch3d = convex_hull_image(mask)

    mask_roi = np.zeros(np.shape(mask))
    chull_3d = np.zeros(np.shape(mask))
    for z in range(0, np.shape(mask)[2]):
        chull_3d[:,:,z] = np.logical_and(chull, ch3d[:,:,z])
        mask_roi[:,:,z] = np.logical_and(mask[:,:,z], chull_3d[:,:,z]) #select the region of interest

    mask_roi = (mask_roi*1).astype('uint8')
    chull_3d = (chull_3d*1).astype('uint8')

    print('skeletonization')
    skeleton = skeletonize_3d(mask_roi)
    skeleton = skeleton.astype('uint8')

    #compute the vascular density and avascular area
    mask[chull_3d==0] = 100 #ignore the area outside the ROI
    total_area = len(chull_3d[chull_3d==1]) #total area of the ROI
    vasc_dens = (len(mask[mask==1]) / (total_area) ) * 100 #vascular density
    avas_area = (len(mask[mask==0]) / (total_area) ) * 100 #avascular area

    #features extracted based on the skeleton using the skan package
    branch_data = summarize(Skeleton(skeleton, spacing=[1,1,1]))
    bpoints, bids, epoints, eids = branching_points(branch_data)
    
    print('Skeletons Features Computed')
    mask[chull_3d==0] = 0
    mask = (mask*1).astype('uint8')

    ellipsoid = draw.ellipsoid(1,1,1, spacing=(1,1,1), levelset=False)
    ellipsoid = ellipsoid.astype('uint8')

    er = binary_erosion(mask, ellipsoid)
    boundaries = mask - er

    major_axes_final, minor_axes_final, radii_final = radius_3d_main(skeleton, branch_data, mask, boundaries)
    
    #add the features to the pandas dataframe
    vessel_features.loc[i] = [msk_name, msk_name.split('_')[0], bpoints/total_area, 
                                      vasc_dens, avas_area, 
                                     (branch_data['branch-distance']).mean(), np.mean(radii_final)]
    i +=1
    
vessel_features.to_csv('features3d.csv', index=False, sep=';')  #um and concave hull do gt, masks cycçlegan 10092022,postprocessed

print(vessel_features)